In [ ]:
import numpy as np
import pandas as pd

mar_26 = pd.read_csv('2024_03_26_reports_peter.csv')

In [ ]:
# Current output
mar_26['MMET Increase'].sum()

Based on the current equation, this should just be the total active miles traveled (without the route shift fraction) * MMET per hour / miles per hour

Constants from 'constants.js' (in the tool github):
1. Route shift constant
  - bicycling: 58.81,
  - walking: 7.73,
2. MMET per hour constant
  - bicycling:
    - lower: 2.5,
    - mean: 4.8,
    - upper: 9.0,
  - walking:
    - lower: 2.0,
    - mean: 2.5,
    - upper: 7.0,
3. miles per hour (speed) constant
  - bicycling:
    - lower: 5.5,
    - mean: 10.0,
    - upper: 15.0,
  - walking:
    - lower: 1.5,
    - mean: 3.0,
    - upper: 4.5,

In [ ]:
# Total increase in active transportation miles traveled
(mar_26['BMT (Total)']+mar_26['WMT (Total)']).sum()

In [ ]:
# Bike total added travel
mar_26['BMT (Total)'].sum()

In [ ]:
# Bike non-route shift added travel
((mar_26['BMT (Total)']).sum())*(100-58.81)/100

In [ ]:
# Bike MMET
((mar_26['BMT (Total)']).sum())*((100-58.81)/100)*(4.8/10)

In [ ]:
# Walk total added travel
mar_26['WMT (Total)'].sum()

In [ ]:
# Walk non-route shift added travel
((mar_26['WMT (Total)']).sum())*(100-7.73)/100

In [ ]:
# Walk MMET
((mar_26['WMT (Total)']).sum())*((100-7.73)/100)*(2.5/3)

In [ ]:
# Total MMET
(((mar_26['BMT (Total)']).sum())*(100-58.81)/100*4.8/10) + (((mar_26['WMT (Total)']).sum())*(100-7.73)/100*2.5/3)

Wow, this is MUCH lower. What is the difference?

In [ ]:
(mar_26['MMET Increase'].sum())/(((mar_26['BMT (Total)']).sum())*(100-58.81)/100*4.8/10) + (((mar_26['WMT (Total)']).sum())*(100-7.73)/100*2.5/3)

43,000 times larger?! Something definitely seems very off here...

Let's calculate individually for each project and compare to the actual output

In [ ]:
mar_26['MMET_test_walk'] = mar_26['WMT (Total)']*((100-7.73)/100)*(2.5/3)
mar_26['MMET_test_bike'] = mar_26['BMT (Total)']*((100-58.81)/100)*(4.8/10)
mar_26['MMET_test_total'] = mar_26['MMET_test_walk']+mar_26['MMET_test_bike']

In [ ]:
# Current calculations
mar_26['MMET_test_total'].plot()

In [ ]:
# Tool output
(mar_26['MMET Increase']).plot()

The shape of the graph is basically identical, the scale is completely different. It looks pretty likely that there is some constant or conversion factor that is different between them.

In [ ]:
# Is it a constant ratio?
mar_26['MMET Increase']/mar_26['MMET_test_total']

In [ ]:
(mar_26['MMET Increase']/mar_26['MMET_test_total']).describe()

Yes, the tool output is exactly 5158.887881 times larger than my calculations.

In [ ]:
# Guess: maybe part of this is being scaled over the 20 year project time frame?
def discount(x):
    discounted = 0
    for i in range(1,21):
        discounted += x/pow(1.04,i-1)
    return(discounted)
mar_26['MMET_test_total_20year']= discount(mar_26['MMET_test_total'])

In [ ]:
# New ratio:
(mar_26['MMET Increase']/mar_26['MMET_test_total_20year'])

Oh, the new ratio is 365. So it looks like the constant is the conversion from the MMET in one day to one year and then to 20 years (with discount rate).